# Downloading and Using ERA-5 Data
Besides the GFS, there's another source of historical open source data we can make use of - [ECMWF's ERA-5 land dataset](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview) This dataset is a 0.1° resolution going back to January 1950. It's got lots of features that we can use for training.

Something to think about, is that the <span style="color:red">**ERA-5 dataset contains observations only**</span>. This means if we predict a wildfire in the future at time `t+x` based on this set, there's no forecasts available for that time, only observations (well, reanalysis) of what actually happened. This is something that will need to be thought about. How will this impact your predictions? Renanalyis forecasts are not the same as leadtime forecasts, where you try to predict the future.

We can download ERA5 data by following [these instructions](https://confluence.ecmwf.int/display/CKB/How+to+download+ERA5). You'll need to register for another account.

With some luck, the `venv` you've built from the `readme` is setup with the `cdsapi` [module](https://anaconda.org/conda-forge/cdsapi). This means you can skip the `pip` installer step mentioned in [these instructions](https://confluence.ecmwf.int/display/CKB/How+to+install+and+use+CDS+API+on+macOS).

In [89]:
import cdsapi
import calendar
import logging
from pathlib import Path
import pandas as pd
from typing import List, Optional
from utils import setup_logging
from functools import partial

_ = setup_logging(debug=False)

## Making a request for data

Here's an example for downloading 2 years of grib files in 6 hrly windows. We need to trade off between temporal resolution and data size. To make your own API calls, see [here](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=form).

The example below is a script that can download weather data in a format known as GRIB. It selects some fields of interest, and crops it over the range of the US before download through the API. I'm only gathering an analysis every 6 hours.

**<span style="color:red">**Note: this will take quite some time to download**</span> , and will depend heavily on your temporal resolution, number of fields, defined area, and the demand on ECMWF's servers at the time.** It may be time for another coffee... ☕️

You may want to experiment with only a few months and lots of features initially, perform some correlation studies (I'm a big fan of shap analyses) and then use this to narrow down to a select set of features with strong importance values. - Or maybe downsample to a daily/weekly forecast over a long range - it's up to you!

In [90]:
download_path = Path("/Users/alex/gitrepos/IRP-help/data")

year_list = [2018]
month_list = [1, 2]


for year in year_list:
    for month in month_list:
        file_name = f"ERA5L_{year:04d}_{month:02d}.grib"
        _, num_of_days = calendar.monthrange(year, month)
        day_list = [f"{day:02d}" for day in range(1, num_of_days + 1)]
        logging.info("Fetching data for YEAR: %s, MONTH: %s", year, month)
        c = cdsapi.Client()

        _ = c.retrieve(
            "reanalysis-era5-land",
            {
                "variable": [
                    "10m_u_component_of_wind",
                    "10m_v_component_of_wind",
                    "2m_dewpoint_temperature",
                    "2m_temperature",
                    "evaporation_from_vegetation_transpiration",
                    "leaf_area_index_high_vegetation",
                    "leaf_area_index_low_vegetation",
                    "skin_reservoir_content",
                    "skin_temperature",
                    "snow_depth",
                    "snowmelt",
                    "soil_temperature_level_1",
                    "surface_latent_heat_flux",
                    "total_precipitation",
                    "volumetric_soil_water_layer_1",
                ],
                "year": f"{year:04d}",
                "month": f"{month:02d}",
                "day": day_list,
                "time": [
                    "00:00",
                    "06:00",
                    "12:00",
                    "18:00",
                ],
                "area": [
                    49,
                    -125,
                    25,
                    -67,
                ],
                "format": "grib",
            },
            f"{str(download_path / file_name)}",
        )

[2023-04-07T16:49:30Z] INFO: Fetching data for YEAR: 2018, MONTH: 1
2023-04-07 16:49:31,663 INFO Welcome to the CDS
[2023-04-07T16:49:31Z] INFO: Welcome to the CDS
2023-04-07 16:49:31,665 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
[2023-04-07T16:49:31Z] INFO: Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-04-07 16:49:31,907 INFO Request is queued
[2023-04-07T16:49:31Z] INFO: Request is queued
2023-04-07 16:49:33,066 INFO Request is running
[2023-04-07T16:49:33Z] INFO: Request is running
2023-04-07 16:55:52,479 INFO Request is completed
[2023-04-07T16:55:52Z] INFO: Request is completed
2023-04-07 16:55:52,481 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data6/adaptor.mars.internal-1680886172.4418323-5940-9-bfe8a6e7-17db-4a39-9bb0-55d43dfcdf17.grib to /Users/alex/gitrepos/IRP-help/data/ERA5L_2018_01.grib (434.7M)
[2023-04-07T16:55:52Z] INFO:

## Viewing the data

The GRIB files produced by the API aren't exactly standard, and some programmes will stuggle to load them. I'd reccomend checking `panoply` if you want a GUI based system to inspect them, found [here](https://www.giss.nasa.gov/tools/panoply/download/), but bear in mind you'll need to change some settings by following [this page.](https://confluence.ecmwf.int/pages/viewpage.action?pageId=165319287)

You may get some errors if your key hasn't been set-up correctly. From the [copernicus homepage](https://cds.climate.copernicus.eu/cdsapp#!/home) you should see a "*Climate Data Store API*" tab, clikcing on that once logged in will show you a box you need to copy and paste into `~/.cdsapirc`.

If you have issues with this, let me know and we can set up a call to go through it interactively as a group.


Some other datasets from ECMWF that you may want to look into are:
- [Fire danger indices historical data from the Copernicus Emergency Management Service](https://cds.climate.copernicus.eu/cdsapp#!/dataset/cems-fire-historical)

## Opening GRIB files in Python

There are plenty of libraries that open GRIB files in Python, but only a few work reliably. I usually use `xarray` with the `cfgrib` engine, but this seems to struggle with the ECMWF GRIBs from the API. GRIB's are a tricky format. As an example, and something you'll likely discover for yourself, it is technically possible to store both GRIB1 and GRIB2 messages in the same GRIB file, although not a recommended practice.

This can make it tricky to automate the loading of GRIB files, as we'll see below...

In [91]:
import xarray as xr
from pathlib import Path

### Opening a Single File

In [92]:
ds = xr.open_dataset(
    "/Users/alex/gitrepos/IRP-help/data/ERA5L_2018_01.grib",
    engine="cfgrib",
    backend_kwargs=dict(filter_by_keys={"typeOfLevel": "surface", "edition": 1}),
    errors="ignore",
)

[2023-04-07T17:01:47Z] WARNING: Ignoring index file '/Users/alex/gitrepos/IRP-help/data/ERA5L_2018_01.grib.923a8.idx' older than GRIB file


In [93]:
ds.drop_vars(["number", "step", "surface", "valid_time"])

<xarray.Dataset>
Dimensions:    (time: 32, latitude: 241, longitude: 581, step: 4)
Coordinates:
  * time       (time) datetime64[ns] 2017-12-31 2018-01-01 ... 2018-01-31
  * latitude   (latitude) float64 49.0 48.9 48.8 48.7 ... 25.3 25.2 25.1 25.0
  * longitude  (longitude) float64 -125.0 -124.9 -124.8 ... -67.2 -67.1 -67.0
Dimensions without coordinates: step
Data variables:
    u10        (time, step, latitude, longitude) float32 ...
    v10        (time, step, latitude, longitude) float32 ...
    d2m        (time, step, latitude, longitude) float32 ...
    t2m        (time, step, latitude, longitude) float32 ...
    lai_hv     (time, step, latitude, longitude) float32 ...
    lai_lv     (time, step, latitude, longitude) float32 ...
    smlt       (time, step, latitude, longitude) float32 ...
    slhf       (time, step, latitude, longitude) float32 ...
    tp         (time, step, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-04-07T11:01 GRIB to CDM+CF via cfgrib-0.9.1...

loading gribs can be very tricky, here are some resources to help:
- [GitHub Discussion Thread](https://github.com/ecmwf/cfgrib/issues/83)
- [cfgrib presentation](https://www.ecmwf.int/sites/default/files/elibrary/2018/18727-cfgrib-easy-and-efficient-grib-file-access-xarray.pdf)

Setting `errors` `kwarg` as `ignore` will get the dataset built, but bear in mind that his has reduced the available variables, you may want to experiment with filtering by keys, and then joining the data once you get it out into a pandas dataframe. Or maybe filtering by edition (GRIB 1 and 2) - loading them seperately, and then joining them in `xarray`.

### Opening Many GRIB Files
We can also open datasets en-masse using the `open_mfdataset` method

In [115]:
def drop_coords_and_filter_by_fields(dataset: xr.Dataset, fields: List[str]) -> xr.Dataset:
    dataset = dataset.drop_vars(["number", "step", "surface"])
    if fields is not None:
        dataset = dataset[set(dataset.data_vars).intersection(fields)]
    return dataset


preproc_func = partial(drop_coords_and_filter_by_fields, fields=None)

In [116]:
grib_files = Path("/Users/alex/gitrepos/IRP-help/data").glob("ERA5L_*_*.grib")
gribs = [str(pth) for pth in grib_files]

ds_joined = xr.open_mfdataset(
    gribs,
    preprocess=preproc_func,
    parallel=True,
    engine="cfgrib",
    combine="by_coords",
    decode_cf=False,
    compat="broadcast_equals",
    backend_kwargs={"filter_by_keys": {"typeOfLevel": "surface", "edition": 1}},
    errors="ignore",
).squeeze()

ds_joined = xr.decode_cf(ds_joined)

[2023-04-07T18:47:08Z] WARNING: Ignoring index file '/Users/alex/gitrepos/IRP-help/data/ERA5L_2018_02.grib.923a8.idx' older than GRIB file
[2023-04-07T18:47:08Z] WARNING: Ignoring index file '/Users/alex/gitrepos/IRP-help/data/ERA5L_2018_01.grib.923a8.idx' older than GRIB file


In [118]:
df = ds_joined.to_dataframe().reset_index()

In [120]:
df.head()

0          2017-12-31 06:00:00
1          2017-12-31 12:00:00
2          2017-12-31 18:00:00
3          2018-01-01 00:00:00
4          2017-12-31 06:00:00
                   ...        
34165119   2018-03-01 00:00:00
34165120   2018-02-28 06:00:00
34165121   2018-02-28 12:00:00
34165122   2018-02-28 18:00:00
34165123   2018-03-01 00:00:00
Name: valid_time, Length: 34165124, dtype: datetime64[ns]

We can join this to the wildfire dataset to have a dataset. Note: I would recommend cleaning and choosing fields before doing this merge on a large scale. It is quite computationally expensive.

Once you have a decision, you can reach out to me, and I can run the code for you on our supercomputer systems (If you are not able to get a hold of these at Cranfield in a timely manner). I can then send you across the dataset in a desired format.

In [99]:
df_truth = pd.read_parquet("/Users/alex/gitrepos/IRP-help/data/export.parquet.gzip")

df_joined = pd.merge(
    df, df_truth, how="left", left_on=["valid_time", "latitude", "longitude"], right_on=["timestamp", "lat", "lon"]
)

In [100]:
df_joined.head()

,time,latitude,longitude,step,u10,v10,d2m,t2m,lai_hv,lai_lv,...,tp,fire_id,timestamp,lat,lon,fire_size_acres,fire_size_class,fire_code,cause_class,month
0,2017-12-31,49.0,-125.0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-12-31,49.0,-125.0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-12-31,49.0,-125.0,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-12-31,49.0,-125.0,3,-1.01355,-1.50824,272.192719,273.852142,2.75293,0.0,...,6.556511e-07,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-12-31,49.0,-124.9,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_joined = df_joined.drop(["valid_time", "latitude", "longitude"])

In [101]:
df_joined.to_parquet("/Users/alex/gitrepos/IRP-help/data/training_set.parquet.gzip", compression="gzip")

This should work, but bear in mind I've not tested it thoroughly. I would check and use these as examples for undertaking your own data works. Reach out if you come across any issues, I'd be happy to help!